# Profiling

In [ ]:
import cProfile
import pandas as pd
import re
from datetime import datetime
from pathlib import Path
import sys
import os
import numpy as np


notebook_dir = Path().resolve()
pyres_dir = str(notebook_dir.parent) + "/PyRes"
sys.path.append(pyres_dir)
import pyres_fof
os.environ["TPTP"] = str(notebook_dir) + "/TPTP-v9.0.0"

In [ ]:
df_tptp = pd.read_csv("df_tptp.csv", index_col=0)

opts = [
    # ("-s", ""),
    ("-r", "120"),
]

problem_selector = (
    (df_tptp["spc"].str.endswith("NEQ") | df_tptp["spc"].str.contains("CNF"))
    & df_tptp["Problem"].str.contains("PUZ")
    & df_tptp["Problem"].str.contains("PUZ002-1")
    # & df_tptp["n_clauses"].between(3,5)
).fillna(False)

problem_dir = (
    df_tptp.loc[problem_selector, "full_file_name"]
    .reset_index(drop=True)
    [0]
    .replace("TPTP-v9.0.0/", "")
)

problem_name = problem_dir[problem_dir.rindex("/")+1:]
args = [
    problem_dir,
    # "TPTP-v9.0.0/Problems/AGT/AGT042+1.p"
    # pyres_dir + "/EXAMPLES/PUZ001+1.p"
    # pyres_dir + "/EXAMPLES/PUZ002-1.p"
    # "TPTP-v9.0.0/Problems/MED/MED007+1.p"
    # "TPTP-v9.0.0/Problems/CSR/CSR038+3.p"
]

with cProfile.Profile() as pr:
    pyres_fof.main(from_notebook=True, notebook_opts=opts, notebook_args=args)
    df_function_stats = pd.DataFrame(
        pr.getstats(),
        columns=["func", "ncalls", "ccalls",
                 "tottime", "cumtime", "callers"]
    )
df_function_stats["func"] = df_function_stats["func"].astype(str)
df_function_stats = \
    df_function_stats[ \
        df_function_stats["func"].str.contains(r"PyRes/(\w+)\.py") ]

func_names = []
lines = []
for value in df_function_stats["func"]:
    func_name = re.search(r'(?<=code object )(\w+)', value)
    line = re.search(r'(?<=line )(\d+)', value)
    func_names.append(func_name.group() if func_name is not None else None)
    lines.append(int(line.group()) if line is not None else None)
df_function_stats["func_name"] = func_names
df_function_stats["line"] = lines

df_function_stats = (
        df_function_stats[["func_name", "line", "ncalls", "ccalls",
                  "tottime", "cumtime", "func", "callers"]]
        .sort_values("ncalls", ascending=False)
)

datestring = (
    datetime.now().isoformat()
    .replace("2025-", "")
)
datestring = datestring[:datestring.rindex(".")]

df_function_stats.to_csv(
    str(notebook_dir)
    + f"/analysis_data/profiling_csv/{datestring}_{problem_name}.csv"
    .replace(".p", "")
)
df_function_stats

Problems/PUZ/PUZ002-1.p
# rel_distance: 120
# graph_construction_time: 0.0025040000000000617
# neighbourhood_computation_time: 0.0021329999999999405
% SZS status Unsatisfiable

% Initial clauses    : 12
% Processed clauses  : 22
% Factors computed   : 0
% Resolvents computed: 22
% Tautologies deleted: 0
% Forward subsumed   : 0
% Backward subsumed  : 0
% -------- CPU Time ---------
% User time          : 1.042 s
% System time        : 0.282 s
% Total time         : 1.324 s


/var/folders/pb/8wrfvtz5593cjrgr_g89r70m0000gn/T/ipykernel_77522/3918629042.py:42: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_function_stats["func"].str.contains(r"PyRes/(\w+)\.py") ]


,func_name,line,ncalls,ccalls,tottime,cumtime,func,callers
166,edge_neighb_of_subset,80,10038,0,7.007020e-04,7.007020e-04,<code object edge_neighb_of_subset at 0x10fd0f...,None
152,termIsVar,73,5717,0,8.909190e-04,8.909190e-04,"<code object termIsVar at 0x10fbab330, file ""/...",None
177,termIsCompound,81,3709,0,1.227420e-03,6.737140e-04,"<code object termIsCompound at 0x10fd2a880, fi...","[(<code object termIsVar at 0x10fbab330, file ..."
151,__len__,90,1961,0,4.291270e-04,3.071090e-04,"<code object __len__ at 0x10fbab230, file ""/Us...","[(<built-in method builtins.len>, 1961, 0, 0.0..."
153,termFunc,89,1828,0,9.653170e-04,3.567570e-04,"<code object termFunc at 0x10fbab430, file ""/U...","[(<code object termIsCompound at 0x10fd2a880, ..."
...,...,...,...,...,...,...,...,...
162,__init__,86,1,0,3.330000e-07,3.330000e-07,"<code object __init__ at 0x10fd0dbd0, file ""/U...",None
99,addEqAxioms,160,1,0,9.062500e-05,1.445800e-05,"<code object addEqAxioms at 0x108261f50, file ...","[(<code object isPred at 0x10fd2a4c0, file ""/U..."
170,__init__,155,1,0,1.250000e-07,1.250000e-07,"<code object __init__ at 0x10fd29e30, file ""/U...",None
171,__init__,61,1,0,2.500000e-07,2.500000e-07,"<code object __init__ at 0x10fd2a1f0, file ""/U...",None


In [4]:
df_function_stats.sort_values("ncalls", ascending=False)[["func_name", "ncalls"]]

,func_name,ncalls
163,edge_neighb_of_subset,10038
151,termIsVar,5916
174,termIsCompound,3849
150,__len__,1950
152,termFunc,1898
...,...,...
215,__init__,1
216,saturate,1
224,parse,1
211,get_all_nodes,1


#### Filter for alternating path functions 

In [ ]:
(
    df_function_stats
    [ df_function_stats["func"].str.contains("alternatingpath") ]
    .sort_values("ncalls", ascending=False)
    ["cumtime"].sum()
)

np.float64(0.0026481839999999996)